<a href="https://colab.research.google.com/github/oktaviacitra/ai-course-in-graduate-school/blob/main/AlternativePCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive

origin_path = '/content/drive'
drive.mount(origin_path)
folder_path = origin_path + '/MyDrive/Learning Journey/AutoEncoder/'

Mounted at /content/drive


In [24]:
import random
import torch
from   torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from   torchvision import datasets, transforms

IMAGE_SIZE = 784
IMAGE_WIDTH = IMAGE_HEIGHT = 28

class AutoEncoder(nn.Module):
  def __init__(self, code_size):
    super().__init__()
    self.code_size = code_size
    
    # Encoder specification
    self.enc_cnn_1 = nn.Conv2d(1, 10, kernel_size=5)
    self.enc_cnn_2 = nn.Conv2d(10, 20, kernel_size=5)
    self.enc_linear_1 = nn.Linear(4 * 4 * 20, 50)
    self.enc_linear_2 = nn.Linear(50, self.code_size)
    
    # Decoder specification
    self.dec_linear_1 = nn.Linear(self.code_size, 160)
    self.dec_linear_2 = nn.Linear(160, IMAGE_SIZE)

  def forward(self, images):
    code = self.encode(images)
    out = self.decode(code)
    return out, code

  def encode(self, images):
    code = self.enc_cnn_1(images)
    code = F.selu(F.max_pool2d(code, 2))
    code = self.enc_cnn_2(code)
    code = F.selu(F.max_pool2d(code, 2))
    code = code.view(images.size(0), -1)
    code = F.selu(self.enc_linear_1(code))
    code = self.enc_linear_2(code)
    return code

  def decode(self, code):
    out = F.selu(self.dec_linear_1(code))
    out = F.sigmoid(self.dec_linear_2(out))
    out = out.view([code.size(0), 1, IMAGE_WIDTH, IMAGE_HEIGHT])
    return out

# Hyperparameters
code_size = 20
num_epochs = 5
batch_size = 128
lr = 0.002
optimizer_cls = optim.Adam

# Load data
train_data = datasets.MNIST('~/data/mnist/', train=True, download=True, transform=transforms.ToTensor())
test_data  = datasets.MNIST('~/data/mnist/', train=False, download=True, transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=batch_size, num_workers=4, drop_last=True)

# Instantiate model
autoencoder = AutoEncoder(code_size)
loss_fn = nn.BCELoss()
optimizer = optimizer_cls(autoencoder.parameters(), lr=lr)

# Training loop
for epoch in range(num_epochs):
  print("Epoch %d" % epoch)
  for i, (images, _) in enumerate(train_loader):
    out, code = autoencoder(Variable(images))
    optimizer.zero_grad()
    loss = loss_fn(out, images)
    loss.backward()
    optimizer.step()
  print("Loss = %.3f" % loss.data)

Epoch 0
Loss = 0.112
Epoch 1
Loss = 0.101
Epoch 2
Loss = 0.103
Epoch 3
Loss = 0.099
Epoch 4
Loss = 0.095


In [35]:
# Try reconstructing on test data
test_image = random.choice(test_data)
test_image = Variable(test_image[0].view(1, 1, IMAGE_WIDTH, IMAGE_HEIGHT))
test_reconst, _ = autoencoder(test_image)

torchvision.utils.save_image(test_image.data, folder_path + 'orig.png')
torchvision.utils.save_image(test_reconst.data, folder_path + 'reconst.png')

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
